# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)

In [20]:
!pip install sklearn

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


telco_url = 'https://raw.githubusercontent.com/will-cotton4/DS-Unit-4-Sprint-2-Neural-Networks/master/module4-Hyperparameter-Tuning/telcochurn.csv'
telco = pd.read_csv(telco_url)

def clean(df):
  df['gender'] = df['gender'].replace({'Male': 1, 'Female': 0})
  df['Partner'] = df['Partner'].replace({'Yes': 1, 'No': 0})
  df['Dependents'] = df['Dependents'].replace({'Yes': 1, 'No': 0})
  df['PhoneService'] = df['PhoneService'].replace({'Yes': 1, 'No': 0})
  df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 0, 'No': 0, 'Yes': 1})
  
  df['Fiber'] = df['InternetService'] == 'Fiber optic'
  df['Fiber'] = df['Fiber'].astype(int)
  df['DSL'] = df['InternetService'] == 'DSL'
  df['DSL'] = df['DSL'].astype(int)
  df['No Internet'] = df['InternetService'].where(df['InternetService'] == 'No', 0)
  df['No Internet'] = df['No Internet'].replace({'No': 1})
  
  df['OnlineSecurity'] = df['OnlineSecurity'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  df['OnlineBackup'] = df['OnlineBackup'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  df['DeviceProtection'] = df['DeviceProtection'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  df['TechSupport'] = df['TechSupport'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  df['StreamingTV'] = df['StreamingTV'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  df['StreamingMovies'] = df['StreamingMovies'].replace({'Yes': 1, 'No': 0,'No internet service': 0})
  
  df['Month to month'] = df['Contract'] == 'Month-to-month'
  df['Month to month'] = df['Month to month'].astype(int)
  df['Two year'] = df['Contract'] == 'Two year'
  df['Two year'] = df['Two year'].astype(int)
  df['One year'] = df['Contract'] == 'One year'
  df['One year'] = df['One year'].astype(int)
  
  yearly_condition = ('One year' or 'Two year')
  df['One or two year'] = df['Contract'] == yearly_condition
  df['One or two year'] = df['One or two year'].astype(int)
  
  df['Electronic check'] = df['PaymentMethod'] == 'Electronic check'
  df['Electronic check'] = df['Electronic check'].astype(int)
  df['Mailed check'] = df['PaymentMethod'] == 'Mailed check'
  df['Mailed check'] = df['Mailed check'].astype(int)
  
  check_condition = ('Mailed check' or 'Electronic check')
  df['pay by check'] = df['PaymentMethod'] == check_condition
  df['pay by check'] = df['pay by check'].astype(int)
  
  df['Bank transfer'] = df['PaymentMethod'] == 'Bank transfer (automatic)'
  df['Bank transfer'] = df['Bank transfer'].astype(int)
  df['Credit card'] = df['PaymentMethod'] == 'Credit card (automatic)'
  df['Credit card'] = df['Credit card'].astype(int)
  
  auto_pay_condition = ('Bank transfer' or 'Credit card')
  df['Auto_pay'] = df['PaymentMethod'] == auto_pay_condition
  df['Auto_pay'] = df['Auto_pay'].astype(int)
  
  df['PaperlessBilling'] = df['PaperlessBilling'].replace({'Yes': 1, 'No': 0})
  df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
  df['TotalCharges'] = df['TotalCharges'].replace({np.nan:0})
  df['Churn'] = df['Churn'].replace({'No': 0, 'Yes':1})
  df = df.drop(columns = ['customerID', 'Contract', 'PaymentMethod', 'InternetService'])
  
  X = df.drop(columns = ['Churn'])
  y = df['Churn']
  return X, y

X, y = clean(telco)

from tensorflow.keras.utils import to_categorical

# y = to_categorical(y, 2)

X = X.values
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)



gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
Fiber               0
DSL                 0
No Internet         0
Month to month      0
Two year            0
One year            0
One or two year     0
Electronic check    0
Mailed check        0
pay by check        0
Bank transfer       0
Credit card         0
Auto_pay            0
dtype: int64


- Create and fit a baseline Keras MLP model to the data.


In [39]:
print(y_train.shape)

(5282, 2)


In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(30, input_dim=29, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=150, verbose=0)

scores = model.evaluate(X_train, y_train)
print("train ", scores)
scores = model.evaluate(X_test, y_test)
print("test ", scores)

5282/5282 [==============================] - 0s 46us/sample - loss: 1.7877 - acc: 0.5186
train  [1.7876619215418514, 0.51855356]
1761/1761 [==============================] - 0s 27us/sample - loss: 1.7766 - acc: 0.5207
test  [1.776557861709378, 0.52072686]


- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?

In [0]:
def create_model():
  model = Sequential()
  model.add(Dense(30, input_dim=29, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(25, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [53]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {"batch_size":[10, 50, 100],
              "epochs": [20],
              "optimizer": ['adam', 'SGD'],
              "optimizer__lr": [0.001, 0.1],
              "optimizer__momentum": [0, 0.01],
              "activation": ['relu', 'sigmoid']
}

Using TensorFlow backend.


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?